## Clean And Analyze Employee Exit Surveys
---------------------------------------------

In this project we will clean the data and analyze employee exit surveys of Department of Education, Training and Employment (**DETE**) and the Technical and Further Education (**TAFE**) institute in Queensland, Australia.

We will try to answer the following questions:

- Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction? What about older employees?

You can find the TAFE exit survey [here](https://data.gov.au/dataset/ds-qld-89970a3b-182b-41ea-aea2-6f9f17b5907e/details?q=exit%20survey) and the survey for the DETE [here](https://data.gov.au/dataset/ds-qld-fe96ff30-d157-4a81-851d-215f2a0fe26d/details?q=exit%20survey).

In [1]:
# IMPORTS
import pandas as pd
import numpy as np

# SETTINGS
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# Read data files
dete_survey = pd.read_csv("dete_survey.csv")
tafe_survey = pd.read_csv("tafe_survey.csv", engine='python')

#### Exploring data sets
- Both data sets have null values. Most of the columns have few to 200 null values and few columns have nearly all null values
- Range of column values are as below
    - Yes/No
    - True/False
    - Strongly Agree/Agree..../Strongly Disagree/NA (both in short form and abbreviated form)
    - Few custom options to choose from

In [19]:
# ... work in progress. Will be updated accordingly.